In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import datetime
import cv2
from keras import backend as K
from keras.layers import Layer,InputSpec
from keras import layers, models
import keras.layers as kl
from glob import glob
from sklearn.metrics import roc_curve, auc
from keras.preprocessing import image
from tensorflow.keras.models import Sequential
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from tensorflow.keras import callbacks 
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from  matplotlib import pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.layers import concatenate, Dense, Conv2D, MaxPooling2D, Flatten, Input, Activation, add, AveragePooling2D, BatchNormalization, Dropout
%matplotlib inline
import shutil
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
from tensorflow.python.platform import build_info as tf_build_info
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
print(os.getcwd())
os.chdir("..")
print(os.getcwd())

D:\Github\Stomach-Status-Classification\Data Preprocessing
D:\Github\Stomach-Status-Classification


In [3]:
main_data_dir = os.getcwd() + "\\Data set"
dl_data_dir = main_data_dir + "\\DL_data"
aug_dl_data_dir = main_data_dir + "\\aug_dl_data"
if not os.path.exists(aug_dl_data_dir):
    os.mkdir(aug_dl_data_dir)
print(os.listdir(dl_data_dir))

['Abnormal', 'Normal']


In [4]:
ab_aug_dir = aug_dl_data_dir + "\\Abnormal"
no_aug_dir = aug_dl_data_dir + "\\Normal"

if not os.path.exists(ab_aug_dir):
    os.mkdir(ab_aug_dir)
if not os.path.exists(no_aug_dir):
    os.mkdir(no_aug_dir)

In [5]:
print(os.listdir(aug_dl_data_dir))

['Abnormal', 'Normal']


In [6]:
img_paths = glob(dl_data_dir + "\\*\\*")
print(img_paths[0])

D:\Github\Stomach-Status-Classification\Data set\DL_data\Abnormal\CHGastro_Abnormal_012.png


In [7]:
target_size = (224, 224)
def Get_data(img_paths, _target_size):
    img_list = []
    labels = []

    for x in img_paths:
        img_list.append(cv2.imread(x))
        labels.append(x.split("\\")[-2])
    for x in range(len(img_list)):
        img_list[x] = tf.keras.preprocessing.image.smart_resize(img_list[x], _target_size)
    
    return (img_list, labels)

In [8]:
img_stack = Get_data(img_paths, target_size)
print(len(img_stack))
print(len(img_stack[0]))

2
176


In [24]:
data_augmentation = tf.keras.Sequential([
#     layers.Rescaling(1./255),
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomContrast((0.1, 0.9)),
#     tf.keras.layers.RandomTranslation(height_factor=(-0.2, 0.3), width_factor=(-0.2, 0.3))
    tf.keras.layers.RandomCrop(height = 180, width = 180),
    tf.keras.layers.RandomZoom(height_factor = (0.2, 0.3), width_factor = (0.2, 0.3)),
    layers.Resizing(target_size[0], target_size[1])
])

In [25]:
save_path = aug_dl_data_dir + "\\{0}\\{1}"
def augment_data(img_stacks, count, save_path, model):
    index = 0
    for x, y in zip(img_stacks[0], img_stacks[1]):
        for i in range(count):
            img_base = model(x)
            cv2.imwrite(save_path.format(y, f"aug_{y}_{index}.jpg"), img_base.numpy())
            index += 1

In [26]:
augment_data(img_stack, 3, save_path, data_augmentation)